# test program

In [1]:
import time
import numpy as np
from poppyControl import *
import sys
from environment import *

sys.path.insert(1, 'C:/Users/dexte/Documents/GitHub/Quadruped/Code')
from CPG import *

class newCPG(CTRNNQuadruped):
    def get_positions(self,inputs,motors=None):
        degrees=np.degrees(self.step(imu_feedback=0, velocity_feedback=0))/2
        degrees=np.clip(degrees,0,180)
        return degrees
    def step(self, imu_feedback, velocity_feedback):
        """Update the CTRNN for one timestep."""
        #compute neural activations (discrete update of CTRNN)
        net_input = self.weights @ self.outputs + self.biases
        self.activations += self.dt / self.tau * (-self.activations + net_input)
        self.outputs = self.sigmoid(self.activations)  #apply activation function
        #add oscillatory gait modulation
        self.phases += self.dt * self.omega
        oscillation = np.sin(self.phases)
        #compute motor commands (combining CTRNN output and oscillations)
        motor_commands = self.outputs#np.concatenate([self.outputs[0:3],self.outputs[0:3],self.outputs[0:3],self.outputs[0:3]]) + 0.5 * oscillation
        #apply IMU feedback for balance correction (modify hip joints)
        imu_correction = self.Kp_imu * imu_feedback  # Pitch correction
        motor_commands[::3] += imu_correction  # Adjust hips
        #apply velocity feedback for adaptive stride length (modify knees)
        velocity_correction = self.Kp_vel * velocity_feedback
        motor_commands[1::3] += velocity_correction  # Adjust knee motors
        return np.clip(motor_commands, 0, 1)  # Return motor positions (normalized)

In [2]:
env=environment(1,friction=1)
env.reset()

In [63]:
#0,1,2,3,4 leg 1 (4 being foot)
#5,6,7,8,9 leg 2 (9 being foot)
# 10, 11, 12 being hips
# 13 being chest twist
# 14 being chest 
# 15 neck twist
# 16 neck tilt
# 17 18 19 20 left arm
# 21 22 23 24 right arm
cpg=newCPG(num_n=25)
print("commence trial...")
#env.runTrial(cpg,500,1)
print("move")
action=np.zeros((25))
action[21]=-200
env.quad.setPositions(action)
for k in range(100): #update simulation
    p.stepSimulation()
    time.sleep(1./240.)
    basePos, baseOrn = p.getBasePositionAndOrientation(env.robot_id) # Get model position
    p.resetDebugVisualizerCamera( cameraDistance=0.8, cameraYaw=75, cameraPitch=-20, cameraTargetPosition=basePos) # fix camera onto model
print("trial done")


commence trial...
move
trial done


In [4]:
print("move")
action=np.zeros((25))
action[0]=100
env.step(cpg,action,1,step=300)
print("trial done")



move
trial done
